## Recipe Builder Actions Overview

### Saving a File Cell
If you wish to save the contents of a cell, simply run it.  The `%%writefile` command at the top of the cell will write the contents of the cell to the file named at the top of the cell. You should run the cells manually when applicable. However, **pressing any of the actions at the top will automatically run all file cells relevant to the action**.

### Training and Scoring
Press the associated buttons at the top in order to run training or scoring. The training output will be shown below the `evaluator.py` cell and scoring output will be shown below the `datasaver.py` cell. You must run training at least once before you can run scoring. You may delete the output cell(s). Running training the first time or after changing `requirements.txt` will be slower since the dependencies for the recipe need to be installed, but subsequent runs will be significantly faster. If you wish to see the hidden output add `debug` to the end of the output cell and re-run it.

### Creating the Recipe
When you are done editing the recipe and satisfied with the training/scoring output, you can create a recipe from the notebook by pressing `Create Recipe`. You must run scoring at least once before you can create the recipe. After pressing it, you will see a progress bar showing how much time is left for the build to finish. If the recipe creation is successful the progress bar will be replaced by an external link that you can click to navigate to the created recipe.


## Caution!
* **Do not delete any of the file cells**
* **Do not edit the `%%writefile` line at the top of the file cells**

---

#### **Requirements File** (Optional)
Add additional libraries you wish to use in the recipe to the cell below. You can specify the version number if necessary. The file cell below is a **commented out example**.  The file structure is yaml. It adheres to the specification outlined in the [Conda Documentation](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-file-manually). **NOTE:** The name field is not allowed to be overridden in the file.

In [79]:
%%writefile ~/my-workspace/.recipes/recipe-uTQx3d0gG/requirements.txt

#dependencies:
## Conda dependencies should be attempted first
#- statsmodels=0.10.2
#- pip
#- pip:
## Pip installs can be listed next but should only be used when Conda is unavailable
#   - pmdarima==1.5.3


Overwriting /home/asruser/my-workspace/.recipes/recipe-uTQx3d0gG/requirements.txt


Search here for additional libraries https://anaconda.org/. This is the list of main **libraries already in use**:
`python=3.6.7` `scikit-learn` `pandas` `numpy` `data_access_sdk_python`
**Warning: libraries or specific versions you add may be incompatible with the above libraries**.

---

#### **Configuration Files**
List any hyperparameters you wish to use. Specify the dataset(s) and schema(s) that are needed for training/scoring. To find the dataset ids go to the **Data tab** in Adobe Experience Platform or view the **Datasets** folder in the **Notebooks Data tab** on the left. You can also find schema id in the **Notebooks Data tab** under the **Schemas** folder. Each configuration will only be used for its corresponding action. `ACP_DSW_TRAINING_XDM_SCHEMA` and `ACP_DSW_SCORING_RESULTS_XDM_SCHEMA` will only be used after the recipe has been created.

##### Training Configuration

In [80]:
%%writefile ~/my-workspace/.recipes/recipe-uTQx3d0gG/training.conf

{
   "trainingDataSetId": "60792d410e4ed51948bfbb92",
   "ACP_DSW_TRAINING_XDM_SCHEMA": "https://ns.adobe.com/techmarketingdemos/schemas/cabd0bb3118cf73a3a7f761677c2c823925c78aff7b6da2c",
   "tenantId": "_techmarketingdemos"
}

Overwriting /home/asruser/my-workspace/.recipes/recipe-uTQx3d0gG/training.conf


##### Scoring Configuration

In [81]:
%%writefile ~/my-workspace/.recipes/recipe-uTQx3d0gG/scoring.conf

{
   "scoringDataSetId": "60792d69fbdbf11948a484ec",
   "tenantId": "_techmarketingdemos",
   "scoringResultsDataSetId": "6093037f9109781948b88e43"
}

Overwriting /home/asruser/my-workspace/.recipes/recipe-uTQx3d0gG/scoring.conf


**The following configuration parameters are automatically set for you when you train/score:** 
`ML_FRAMEWORK_IMS_USER_CLIENT_ID` `ML_FRAMEWORK_IMS_TOKEN` `ML_FRAMEWORK_IMS_ML_TOKEN` `ML_FRAMEWORK_IMS_TENANT_ID`

---

#### **Training Data Loader File**
Implement the `load` function to load and prepare the training data.

In [82]:
%%writefile ~/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/trainingdataloader.py

import pandas as pd
from datetime import datetime, timedelta
from platform_sdk.dataset_reader import DatasetReader
from .utils import get_client_context

def load(config_properties):
    print("Training Data Load Start")

    #########################################
    # Load Data
    #########################################    
    client_context = get_client_context(config_properties)
    dataset_reader = DatasetReader(client_context, dataset_id=config_properties['trainingDataSetId'])
    # If you do not see any data or would like to expand the default date range, change the following query
    df0 = dataset_reader.read()
    df0 = df0.drop("productListItems", axis=1)
    dropColumns = []
    for column in df0.columns:
        print(column)
        uniqueValues = df0[column].unique()
        if len(uniqueValues) < 2 and uniqueValues[0] == None:
            dropColumns.append(column)
            print("column to drop %s" % column)
        print(uniqueValues)
    df0 = df0.drop(dropColumns, axis = 1)
    df0 = df0.drop("_id",axis=1)
    def last_element(x):
        y=list(x)
        y.reverse()
        try:
            last_index = y.index(1) 
            last_index_skip = last_index+1
            try:
                second_last_index_temp = y[last_index_skip:].index(1)
                return len(y) - (second_last_index_temp + last_index_skip) -1
            except ValueError:
                return 0
        except ValueError:
            return 0
    def ordered(x):
        y=list(x)
        y.reverse()
        try:
            last_index = y.index(1) 
            return True
        except ValueError:
            return False
    df_journeydemarcation = df0.groupby("endUserIDs._experience.mcid.id")['commerce.purchases.value'].agg( {"demarcation":lambda x: last_element(x),"ordered":lambda x: ordered(x)})
    columns = ['commerce.checkouts.value', 'commerce.order.purchaseID',
       'commerce.productListAdds.value', 'commerce.productViews.value',
       'commerce.purchases.value',
       'web.webPageDetails.name', 'web.webPageDetails.pageViews.value',]
    operation = {}
    for column in columns:
        operation[column] = lambda x: list(x)
    df0_list  = df0.groupby(["endUserIDs._experience.mcid.id"]).agg(operation)
    df0_list["endUserIDs._experience.mcid.id"] = df0_list.index
    print(df0_list.columns)
    df0_list_final = df0_list.join(df_journeydemarcation, "endUserIDs._experience.mcid.id")
    final_columns = ['commerce.checkouts.value', 'commerce.order.purchaseID',
       'commerce.productListAdds.value', 'commerce.productViews.value',
       'commerce.purchases.value', 'web.webPageDetails.name',
       'web.webPageDetails.pageViews.value']
    for column in final_columns:
        print(column)
        df0_list_final[column+"past"] = df0_list_final.apply(lambda row: row[column][0:row['demarcation']], axis=1)
        df0_list_final[column+"current"] = df0_list_final.apply(lambda row: row[column][row['demarcation']+1:], axis=1)
    df0_list_final_current_past = df0_list_final.drop(final_columns, axis=1).drop(["endUserIDs._experience.mcid.id","demarcation"],axis=1)
    df0_list_final_current_past["total_past_checkouts"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.checkouts.valuepast"])), axis = 1)
    df0_list_final_current_past["total_current_checkouts"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.checkouts.valuecurrent"])), axis = 1)
    df0_list_final_current_past["total_past_product_adds"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.productListAdds.valuepast"])), axis = 1)
    df0_list_final_current_past["total_current_product_adds"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.productListAdds.valuecurrent"])), axis = 1)
    df0_list_final_current_past["total_past_product_views"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.productViews.valuepast"])), axis = 1)
    df0_list_final_current_past["total_current_product_views"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.productViews.valuecurrent"])), axis = 1)
    df0_list_final_current_past["total_past_purchases"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.purchases.valuepast"])), axis = 1)
    df0_list_final_current_past["total_past_web_visit"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["web.webPageDetails.pageViews.valuepast"])), axis = 1)
    df0_list_final_current_past["total_current_web_visit"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["web.webPageDetails.pageViews.valuecurrent"])), axis = 1)
    df0_list_final_current_past_training = df0_list_final_current_past.drop([ 'commerce.checkouts.valuepast',
       'commerce.checkouts.valuecurrent', 'commerce.order.purchaseIDpast',
       'commerce.order.purchaseIDcurrent',
       'commerce.productListAdds.valuepast',
       'commerce.productListAdds.valuecurrent',
       'commerce.productViews.valuepast', 'commerce.productViews.valuecurrent',
       'commerce.purchases.valuepast', 'commerce.purchases.valuecurrent',
       'web.webPageDetails.namepast', 'web.webPageDetails.namecurrent',
       'web.webPageDetails.pageViews.valuepast',
       'web.webPageDetails.pageViews.valuecurrent'],axis=1)
    timeframe = config_properties.get("timeframe")
    tenant_id = config_properties.get("tenant_id")
    return df0_list_final_current_past_training

Overwriting /home/asruser/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/trainingdataloader.py


---

#### **Scoring Data Loader File**
Implement the `load` function to load and prepare the scoring data.

In [83]:
%%writefile ~/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/scoringdataloader.py

import pandas as pd
from datetime import datetime, timedelta
from .utils import get_client_context
from platform_sdk.dataset_reader import DatasetReader

def load(config_properties):

    print("Scoring Data Load Start")
    client_context = get_client_context(config_properties)
    dataset_reader = DatasetReader(client_context, dataset_id=config_properties['scoringDataSetId'])
    # If you do not see any data or would like to expand the default date range, change the following query
    df0 = dataset_reader.read()
    df0 = df0.drop("productListItems", axis=1)
    dropColumns = []
    for column in df0.columns:
        print(column)
        uniqueValues = df0[column].unique()
        if len(uniqueValues) < 2 and uniqueValues[0] == None:
            dropColumns.append(column)
            print("column to drop %s" % column)
        print(uniqueValues)
    df0 = df0.drop(dropColumns, axis = 1)
    df0 = df0.drop("_id",axis=1)
    def last_element(x):
        y=list(x)
        y.reverse()
        try:
            last_index = y.index(1) 
            last_index_skip = last_index+1
            try:
                second_last_index_temp = y[last_index_skip:].index(1)
                return len(y) - (second_last_index_temp + last_index_skip) -1
            except ValueError:
                return 0
        except ValueError:
            return 0
    def ordered(x):
        y=list(x)
        y.reverse()
        try:
            last_index = y.index(1) 
            return True
        except ValueError:
            return False
    df_journeydemarcation = df0.groupby("endUserIDs._experience.mcid.id")['commerce.purchases.value'].agg( {"demarcation":lambda x: last_element(x)})
    columns = ['commerce.checkouts.value', 'commerce.order.purchaseID',
       'commerce.productListAdds.value', 'commerce.productViews.value',
       'commerce.purchases.value',
       'web.webPageDetails.name', 'web.webPageDetails.pageViews.value',]
    operation = {}
    for column in columns:
        operation[column] = lambda x: list(x)
    df0_list  = df0.groupby(["endUserIDs._experience.mcid.id"]).agg(operation)
    df0_list["endUserIDs._experience.mcid.id"] = df0_list.index
    print(df0_list.columns)
    df0_list_final = df0_list.join(df_journeydemarcation, "endUserIDs._experience.mcid.id")
    final_columns = ['commerce.checkouts.value', 'commerce.order.purchaseID',
       'commerce.productListAdds.value', 'commerce.productViews.value',
       'commerce.purchases.value', 'web.webPageDetails.name',
       'web.webPageDetails.pageViews.value']
    for column in final_columns:
        print(column)
        df0_list_final[column+"past"] = df0_list_final.apply(lambda row: row[column][0:row['demarcation']], axis=1)
        df0_list_final[column+"current"] = df0_list_final.apply(lambda row: row[column][row['demarcation']+1:], axis=1)
    df0_list_final_current_past = df0_list_final.drop(final_columns, axis=1).drop(["demarcation"],axis=1)
    df0_list_final_current_past["total_past_checkouts"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.checkouts.valuepast"])), axis = 1)
    df0_list_final_current_past["total_current_checkouts"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.checkouts.valuecurrent"])), axis = 1)
    df0_list_final_current_past["total_past_product_adds"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.productListAdds.valuepast"])), axis = 1)
    df0_list_final_current_past["total_current_product_adds"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.productListAdds.valuecurrent"])), axis = 1)
    df0_list_final_current_past["total_past_product_views"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.productViews.valuepast"])), axis = 1)
    df0_list_final_current_past["total_current_product_views"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.productViews.valuecurrent"])), axis = 1)
    df0_list_final_current_past["total_past_purchases"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["commerce.purchases.valuepast"])), axis = 1)
    df0_list_final_current_past["total_past_web_visit"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["web.webPageDetails.pageViews.valuepast"])), axis = 1)
    df0_list_final_current_past["total_current_web_visit"] = df0_list_final_current_past.apply(lambda row: sum(filter(None, row["web.webPageDetails.pageViews.valuecurrent"])), axis = 1)
    df0_list_final_current_past_training = df0_list_final_current_past.drop([ 'commerce.checkouts.valuepast',
       'commerce.checkouts.valuecurrent', 'commerce.order.purchaseIDpast',
       'commerce.order.purchaseIDcurrent',
       'commerce.productListAdds.valuepast',
       'commerce.productListAdds.valuecurrent',
       'commerce.productViews.valuepast', 'commerce.productViews.valuecurrent',
       'commerce.purchases.valuepast', 'commerce.purchases.valuecurrent',
       'web.webPageDetails.namepast', 'web.webPageDetails.namecurrent',
       'web.webPageDetails.pageViews.valuepast',
       'web.webPageDetails.pageViews.valuecurrent'],axis=1)
    timeframe = config_properties.get("timeframe")
    tenant_id = config_properties.get("tenant_id")
    return df0_list_final_current_past_training

Overwriting /home/asruser/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/scoringdataloader.py


---

#### **Pipeline File**
Implement the `train` function and return the trained model. Implement the `score` function to return a prediction made on the scoring data.

In [84]:
%%writefile ~/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/pipeline.py

from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm

def train(config_properties, data):

    print("Train Start")

    X_train = data[0]
    y_train = data[1]

    seed = 1234
    clf = svm.SVC()

    clf.fit(X_train,y_train)
    return clf

def score(config_properties, data, model):

    predict = model.predict(data.drop("endUserIDs._experience.mcid.id",axis=1))
    data["ordered"] = predict
    return data

Overwriting /home/asruser/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/pipeline.py


---

#### **Evaluator File**
Implement the `split` function to partition the training data and the `evaluate` function to the return the validation metrics you wish to see. Training output will be shown below this file cell.

In [85]:
%%writefile ~/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/evaluator.py

from ml.runtime.python.core.regressionEvaluator import RegressionEvaluator
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,auc,f1_score,precision_recall_fscore_support

class Evaluator(RegressionEvaluator):
    def __init__(self):
        print ("Initiate")

    def split(self, config={}, dataframe=None):
        y = dataframe["ordered"]
        x = dataframe.drop("ordered", axis=1)
        train_X, test_X, train_y, test_y = train_test_split(x, 
                                       y,
                                       test_size = 500)
        train_start = '2010-02-12'
        train_end = '2012-01-27'
        val_start = '2012-02-03'
        train = dataframe[train_start:train_end]
        val = dataframe[val_start:]

        return (train_X,train_y),(test_X, test_y)

    def evaluate(self, data=[], model={}, config={}):
        print ("Evaluation evaluate triggered")
        x=data[0]
        y=data[1]
        predict = model.predict(x)
        acc = accuracy_score(y, predict)
        print("accuracy is %f"%acc)
        print("auc is %f"%auc(y, predict))
        x=precision_recall_fscore_support(y, predict, average='binary')
        print("precision is %f recall is %f f1 is %f"%(x[0],x[1],x[2]))

        metric = [{"name": "precision", "value": x[0], "valueType": "double"},
                  {"name": "recall", "value": x[1], "valueType": "double"},
                  {"name": "f1", "value": x[2], "valueType": "double"},
                 {"name": "accuracy", "value": acc, "valueType": "double"}]
        
        print(metric)
        return metric

Overwriting /home/asruser/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/evaluator.py


In [78]:
!bash -e run_action.sh recipe-uTQx3d0gG training

RUNNING TRAINING...
Ignore the exception for python runtime cannot import name 'BlockBlobService'
2021-05-12 22:15:04,057 INFO trainingInitiator.py: Trainer initiated
INFO:trainingInitiator.py:Trainer initiated
2021-05-12 22:15:04,081 INFO trainingInitiator.py: Evaluator initiated
INFO:trainingInitiator.py:Evaluator initiated
2021-05-12 22:15:04,081 INFO main.py: Training starts, testing:True, conf:/tmp/tmp.wMbt2m9HFY/token.conf
INFO:main.py:Training starts, testing:True, conf:/tmp/tmp.wMbt2m9HFY/token.conf
2021-05-12 22:15:04,086 INFO trainingInitiator.py: Training class is not of type Tensorflow
INFO:trainingInitiator.py:Training class is not of type Tensorflow
2021-05-12 22:15:04,087 INFO trainingInitiator.py: Python Job
INFO:trainingInitiator.py:Python Job
2021-05-12 22:15:04,087 INFO trainingInitiator.py: Load the dataframe
INFO:trainingInitiator.py:Load the dataframe
Training Data Load Start
INFO:PlatformSDKPython:dataset_reader: seconds taken to get dataset details from catalog 

---

#### **Data Saver File**
Implement the `save` function for saving your prediction. Scoring output will be added below this cell.

In [86]:
%%writefile ~/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/datasaver.py

import pandas as pd
from .utils import get_client_context
from platform_sdk.models import Dataset
from platform_sdk.dataset_writer import DatasetWriter
from datetime import datetime

def save(config_properties, prediction):
  print("Datasaver Start")

  client_context = get_client_context(config_properties)
  tenant_id = config_properties.get("tenantId")
  prediction[tenant_id+".willOrder"] = prediction["ordered"]
  final_prediction = prediction[[tenant_id+".willOrder"]]
  final_prediction["_techmarketingdemos.ecid"] = final_prediction.index
  now = datetime.now()
 
  # print("now =", now)

  # dd/mm/YY H:M:S
  dt_string = now.strftime("%Y-%m-%dT%H:%M:%S")
  final_prediction = final_prediction.join(pd.DataFrame(
      {
          '_id': "",
          'timestamp': dt_string,
          'eventType': ""
      }, index=final_prediction.index))
    
  final_prediction = final_prediction[['_techmarketingdemos.ecid', '_techmarketingdemos.willOrder']]
  print("head started")
  print(final_prediction.head())

  dataset = Dataset(client_context).get_by_id(config_properties['scoringResultsDataSetId'])
  dataset_writer = DatasetWriter(client_context, dataset)
  dataset_writer.write(final_prediction, file_format='json')

  print("Datasaver Finish")
  print(prediction)

Overwriting /home/asruser/my-workspace/.recipes/recipe-uTQx3d0gG/recipe/datasaver.py


In [87]:
!bash -e run_action.sh recipe-uTQx3d0gG scoring

RUNNING SCORING...
Ignore the exception for python runtime cannot import name 'BlockBlobService'
2021-05-12 22:16:09,324 INFO main.py: Scoring starts, testing:True, conf:/tmp/tmp.QeXrcKjnY4/token.conf
INFO:main.py:Scoring starts, testing:True, conf:/tmp/tmp.QeXrcKjnY4/token.conf
2021-05-12 22:16:12,197 INFO scoringInitiator.py: Scorer initiated
INFO:scoringInitiator.py:Scorer initiated
2021-05-12 22:16:12,198 INFO scoringInitiator.py: Scoring class is not of type Tensorflow
INFO:scoringInitiator.py:Scoring class is not of type Tensorflow
2021-05-12 22:16:12,199 INFO scoringInitiator.py: Python scoring starts
INFO:scoringInitiator.py:Python scoring starts
2021-05-12 22:16:12,199 INFO scoringInitiator.py: Python executed scoring
INFO:scoringInitiator.py:Python executed scoring
Scoring Data Load Start
INFO:PlatformSDKPython:dataset_reader: seconds taken to get dataset details from catalog and make PQS connection: 1.61
INFO:PlatformSDKPython:dataset_id: 60792d69fbdbf11948a484ec, limit: 500